In [1]:
import PEPit

In [2]:
from PEPit import *
from PEPit.functions import SmoothStronglyConvexFunction

In [5]:
def pseudograd_grad_ratio(L, mu, gamma, beta, c0, eta, NCG_PEP_ratio, verbose=1):
    
    n = 1
    
    # Instantiate PEP
    problem = PEP()
    
    # Declare a smooth strongly convex function
    f = problem.declare_function(SmoothStronglyConvexFunction, mu=mu, L=L)
    
    # Start by defining its unique optimal point xs = x_* and 
    # corresponding function value fs = f_*
    xs = f.stationary_point()
    fs = f.value(xs)
    
    # Then define the starting point x0 of the algorithm as well as corresponding gradient and function value g0 and f0
    x0 = problem.set_initial_point()
    g0, f0 = f.oracle(x0)
    
    d0 = Point()
    d_new = d0
    x_new = x0
    g_new = g0
    
    problem.add_constraint((d0) ** 2 == c0*(g0) ** 2)

    problem.add_constraint((d0) ** 2 >= (g0) ** 2)

    problem.add_constraint(g0 * d0 == (g0) ** 2)
    
    for i in range(n):
        print(i)
        d_old = d_new
        x_old = x_new
        g_old = g_new
        x_new = x_old - gamma*d_old
        g_new = f.gradient(x_new)
        d_new = g_new + beta*d_old
        problem.add_constraint(g_new * d_old == 0)
        problem.add_constraint(g_new * (x_old-x_new) == 0)
        problem.add_constraint(g_old * d_old == g_old ** 2)
        problem.add_constraint(beta * g_old ** 2 == g_new **2 - eta * (g_new * g_old))
        
      
    # Set the initial constraint that is the difference between f0 and f_*
    problem.set_initial_condition(g_new**2 == 1)
    
    # Set the performance metric to the function value accuracy
    problem.set_performance_metric(d_new**2)   
    
    # Solve the PEP
    verbose = 1
    pepit_verbose = max(verbose, 0)
    pepit_rho = problem.solve(verbose=pepit_verbose)
    
    
    # Print conclusion if required
    if verbose != -1:
        print('*** Example file ***')
        print("\tPEPit guarantee: \t\t\t d_k^2 >= {:.6} g_k^2".format(pepit_rho))
        print("\tNCG-PEP guarantee: \t\t\t d_k^2 <= {:.6} g_k^2".format( NCG_PEP_ratio))
        
    return pepit_rho   

In [3]:
## These inputs are coming from last line of "2. Using_the_saved_datasets_Julia.ipynb"
L = 1
mu = 0.5
beta = 0.15876766391979427
gamma = 1.4851485148762182
NCG_PEP_ratio = 1.12500000062546
c0 = 1.01
eta = 1

In [6]:
pseudograd_grad_ratio(L, mu, gamma, beta, c0, eta, NCG_PEP_ratio, verbose=1)

0
(PEPit) Setting up the problem: size of the main PSD matrix: 5x5
(PEPit) Setting up the problem: performance measure is minimum of 1 element(s)
(PEPit) Setting up the problem: Adding initial conditions and general constraints ...
(PEPit) Setting up the problem: initial conditions and general constraints (8 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 1 function(s)
		 function 1 : Adding 6 scalar constraint(s) ...
		 function 1 : 6 scalar constraint(s) added
(PEPit) Compiling SDP
(PEPit) Calling SDP solver
(PEPit) Solver status: optimal (solver: MOSEK); optimal value: 1.1249999959713253
(PEPit) Postprocessing: solver's output is not entirely feasible (smallest eigenvalue of the Gram matrix is: -2.59e-09 < 0).
 Small deviation from 0 may simply be due to numerical error. Big ones should be deeply investigated.
 In any case, from now the provided values of parameters are based on the projection of the Gram matrix onto the cone of symmetric semi-defin

1.1249999959713253